1. Named entity recognition (NER) using spaCy
2. Finding occurrences of interesting key terms and their context windows 

In [1]:
import os, re, string
import re
import pandas as pd
import spacy
from collections import Counter
# For first time downloading: 
# !python -m spacy download en_core_web_sm

# Get text info into a dictionary with keys as the file name. 
# Automatically breaks texts that are longer than 1 million words into two separate dictionary entries (assumes that the total size is less than 2m words)
def getTexts(folder):
    texts = {}
    for file in os.listdir(folder):
        if 'NOTES' not in file: 
            name = file.split('.')[0]
            path = os.path.join(folder,file)
            with open(path,'r') as file: 
                data = file.readlines()
            if len(data) != 0: 
                text = data[0].split(' ')
                length = len(text)
                if length > 1000000: 
                    texts[f'{name}_part1'] = ' '.join(text[:length/2])
                    texts[f'{name}_part2'] = ' '.join(text[length/2:])
                else: 
                    texts[name] = ' '.join(text)
    return texts

In [ ]:
playFiles = getTexts('/Users/amycweng/Digital Humanities/Early-Modern-London/playPartsTXT')
print(len(playFiles))

In [2]:
charityTexts = getTexts('/Users/amycweng/Digital Humanities/charityTXT')
print(len(charityTexts))

201


In [ ]:
#running named entity recognition using spaCy
allEnts = []
ner = spacy.load('en_core_web_sm')
count = 0
for name, text in playFiles.items():
    count += 1
    if not count % 40: print(f'processed {count} so far')
    entities = ner(text)
    for ent in entities.ents:
        allEnts.append(str(ent))

In [ ]:
allEntities = []
for ent in allEnts: 
    allEntities.append(str(ent))

In [ ]:
counts = Counter(allEntities)
print(counts.most_common(n=50))

In [ ]:
with open('/Users/amycweng/Digital Humanities/Early-Modern-London/Other_Files/playsNER.txt','w+') as file: 
    file.write(str(counts.most_common(n=10000000)))

In [108]:
places = ['citizen','livery','alderman','pilgrim',
'window','shop','chamber','bridge',
'wall','st paul','westminster','thames',
'river','sea','channel','pond','temple',
'city','village','province','town',
'house',' inn ','market','gate',
'cheapside','bishopsgate','southwark',
'cornhill','tavern','hall','chapel',
'england','hospital','tower','gallow',
'neighbourhood','parish','suburb',
'manor','prison','playhouse',
'churchyard','abbey','aldersgate',
'alley','whitechapel','shire','london',
'company','corporation']
# for idx in range(len(places)): 
#     places[idx] = f'^{places[idx]}$'
place_search = '|'.join(places)

words = ['charity','charitable','alm','bequest',
        'poor','rate','usury','usurer','bequeath',
        'money','beneficence','poverty','credit',
        'creditor','loan','lend','new','novel','novelty',
        'wealth','profit','profitable','beneficial',
        'commodity','thrift','thrifty','industry','bullion',
        'talon','talent','beggar','needy','bestow',
        'liberality','giving','penny','liberal',
        'shilling','pound','score','restitution','default',
        'halfpenny','quarter','ounce','pound','broker',
        'debtor','debt']
# for idx in range(len(words)): 
#     words[idx] = f'^{words[idx]}$'
words_search = '|'.join(words)

In [145]:
phrases = []
for name, text in charityTexts.items():
    indices = []
    text = text.split(' ')
    for idx, word in enumerate(text):
        toContinue = False
        # if re.match('cheapside|southwark|westminster|cornhill|lombard|the tower|thames|bankside|royal exchange|^gate$|whitechapel|hall|bridge|london',word):
        if re.match('citizen|citizenship|alderman',word):  
            for i in indices:
                if idx in range(i-20,i+20):
                    toContinue = True
            if not toContinue: 
                phrases.append((name, ' '.join(text[idx-40:idx+40])))
                indices.append(idx)

In [148]:
count = 0
for phrase in phrases: 
    if re.search('charity|poor|charitable|works of charity|alm',phrase[1]): 
        count += 1
        # print(phrase,'\n')
print(count)

23


In [19]:
phrases = []
for name, text in charityTexts.items():
    indices = []
    text = text.split(' ')
    for idx, word in enumerate(text):
        toContinue = False
        if re.search(r'\bpenny\b|\bpound\b|\bshilling\b|\bgive alm\b|\bhalfpenny\b|\bpence\b',word):  
            for i in indices:
                if idx in range(i-10,i+10):
                    toContinue = True
            if not toContinue: 
                phrases.append((name, ' '.join(text[idx-10:idx+10])))
                indices.append(idx)

In [27]:
for phrase in phrases: 
    if re.search('pope',phrase[1]):
        print(phrase)

('A19588', 'weekly to the pope a julio a piece about six penny sterling the whole revenue whereof in the year do')
('A19589', 'weekly to the pope a julio a piece about six penny sterling the whole revenue whereof in the year do')
